In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#!/usr/bin/env python
# coding: utf-8


In [ ]:
#For data processing
import pandas as pd
import numpy as np
import datetime as dt

#For data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#For models
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# IMPORTING IMPORTANT LIBRARIES
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
import preprocessing 


In [ ]:

# # fix_yahoo_finance is used to fetch data 
# import yfinance as yf
# yf.pdr_override()


# # input
# symbol = 'RELIANCE.NS'
# start = '2017-01-01'
# end = '2018-01-01'


# # dataframe
# dataset = yf.download(symbol,start,end)



# # View the last 5 rows
# print('Last 5 Rows')
# print(df.tail())

#Read input file
df = pd.read_csv("C:\\Users\\tcgra\\Documents\\LJMU_Code\Datasets\\infosys_stock.csv")
df.head(5)



In [ ]:
df.info()

In [ ]:
df['Open'].plot(figsize=(16,6))
df.rolling(window=30).mean()['Open'].plot()


FEATURE GENERATION

In [ ]:
def get_technical_indicators(data): #function to generate feature technical indicators
    import talib as ta
    #Simple Moving Average
    data['SMA'] = ta.SMA(data['Adj Close'], timeperiod = 20)

    # Exponential Moving Average
    data['EMA'] = ta.EMA(data['Adj Close'], timeperiod = 20)
    # Plot
    data[['Adj Close','SMA','EMA']].plot(figsize=(10,5))
    plt.title('Adj Close, SMA & EMA)')
    plt.show()
    
    # Bollinger Bands
    data['upper_band'], data['middle_band'], data['lower_band'] = ta.BBANDS(data['Adj Close'], timeperiod =20)
    # Plot
    data[['Adj Close','upper_band','middle_band','lower_band']].plot(figsize=(10,5))
    plt.show()
    
    # Moving Average Convergence Divergence (MACD)
    data['macd'], data['macdsignal'], data['macdhist'] = ta.MACD(data['Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    data[['macd','macdsignal']].plot(figsize=(10,5))
    plt.title('On Balance Volume (in millions)')
    plt.show() 
        
        
    # Relative Strength Index (RSI)
    data['RSI'] = ta.RSI(data['Adj Close'], timeperiod=14)
    # Plotting RSI
    fig,ax = plt.subplots(figsize=(10,5))
    ax.plot(data.index, data.RSI, label='RSI')
    ax.fill_between(data.index, y1=30, y2=70, color = 'lightcoral', alpha='0.3')
    ax.set_xlabel('Date')
    ax.set_ylabel('RSI')
    plt.show()
    
    # On Balanace Vol (OBV)
    data['OBV'] = ta.OBV(data['Adj Close'], data['Volume'])/10**6
    data.OBV.plot()
    plt.ylabel('On Balance Volume (in millions)')
    plt.show()
    
    #Average Directional Index (ADX)
    data.fillna(0, inplace=True)
#     return dataset
    return data

In [ ]:

df_TI = get_technical_indicators(df)
df_TI.head(10)

In [ ]:
print('Total dataset has {} samples, and {} features.'.format(df_TI.shape[0], \
                                                              df_TI.shape[1]))

FINDING IMPORTANT FEATURES USING XGBOOST

In [ ]:
def get_feature_importance_data(dataset):
    data = dataset.copy()
    y = data['Close']
    X = data.iloc[:,1:19]
    
    train_samples = int(X.shape[0] * 0.65)
 
    X_train = X.iloc[:train_samples]
    X_test = X.iloc[train_samples:]

    y_train = y.iloc[:train_samples]
    y_test = y.iloc[train_samples:]
    
    return (X_train, y_train), (X_test, y_test)

In [ ]:
# Get training and test data
(X_train_FI, y_train_FI), (X_test_FI, y_test_FI) = get_feature_importance_data(df_TI)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

regressor = xgb.XGBRegressor(gamma=0.0,n_estimators=150,base_score=0.7,colsample_bytree=1,learning_rate=0.05)

xgbModel = regressor.fit(X_train_FI,y_train_FI, \
                         eval_set = [(X_train_FI, y_train_FI), (X_test_FI, y_test_FI)], \
                         verbose=False)
eval_result = regressor.evals_result()
training_rounds = range(len(eval_result['validation_0']['rmse']))

In [ ]:
plt.scatter(x=training_rounds,y=eval_result['validation_0']['rmse'],label='Training Error')
plt.scatter(x=training_rounds,y=eval_result['validation_1']['rmse'],label='Validation Error')
plt.xlabel('Iterations')
plt.ylabel('RMSE')
plt.title('Training Vs Validation Error')
plt.legend()
plt.show()

THE DIFFERENCE BETWEEN VALIDATION AND TRAINING ERROR SHOWS A UNDERFIT MODEL. EXPECTED BECAUSE OF THE WAY TRAINING AND TESTING DATA IS SPLIT JUST WHEN STOCK BOOMS

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.xticks(rotation='vertical')
plt.bar([i for i in range(len(xgbModel.feature_importances_))], xgbModel.feature_importances_.tolist(), tick_label=X_test_FI.columns)
plt.title('Infosys: Feature importance in XGBoost.')
plt.show()

We could say that features OHLC and EMA are imp features

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=250, max_depth=25)
y = df_TI['Close']
X = df_TI.iloc[:,1:19]
model.fit(X,y)
importance = pd.DataFrame({'features': X.columns,'importances': model.feature_importances_})
importance = importance.sort_values('importances', ascending=False)
plt.figure(figsize = (16,12))
sns.barplot(importance.importances, importance.features)
plt.title('RandomForestRegressor Feature Importance Plot')
plt.show()

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_FI)

In [ ]:
shap.summary_plot(shap_values, X_test_FI, plot_type="bar")

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
perm_importance = permutation_importance(model, X_test_FI, y_test_FI)

In [ ]:
sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(X.columns[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")